In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=fcd9e762ca21b48af58f45b67d5b68ce45061ef2b08ac4d78665b2a089770f6a
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [10]:
%%writefile hello_world_pycuda.py
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# CUDA kernel code to return data
cuda_code = """
__global__ void return_data(char *output) {
    const char msg[] = "Hello, World!";
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < sizeof(msg)) {
        output[idx] = msg[idx];
    }
}
"""


# Compile the CUDA module
cuda_module = SourceModule(cuda_code)
return_data_kernel = cuda_module.get_function("return_data")

#Allocate memory on the GPU for the output
output_size = 15 # Length of the "Hello, World!" string + null terminator
output_gpu = cuda.mem_alloc(output_size)

# Set up block and grid dimensions
block_dim = (output_size, 1, 1)
grid_dim = (1, 1)

# Launch the kernel
return_data_kernel(output_gpu, block=block_dim, grid=grid_dim)

# Copy the result back to the host
output_host = np.empty(output_size, dtype=np.uint8)
cuda.memcpy_dtoh(output_host, output_gpu)

#Convert the result to a string and print it
output_str = ''.join(chr(c) for c in output_host)
print(output_str)





Overwriting hello_world_pycuda.py


In [11]:
!python hello_world_pycuda.py

Hello, World!  
